In [1]:
// 使用须知
// 1. 用户需要有tmp表的写入权限，申请方法见：https://km.sankuai.com/page/411726480
// 2. 需要增加git权限，将你的公钥添加到https://dev.sankuai.com/code/home 又上角 SSH Key
// 3. 需要升级AFO的版本，（在提交机 /opt/meituan/versions 目录下 下载hdfs上的/user/hadoop-hdp/dushifeng/afo-clients-0.5.70.tar.gz 这个文件， ll /opt/meituan/versions/afo-clients-0.5.70/afo-tensorflow/afo-tensorflow-0.5.70/
// 然后执行 ln -snf /opt/meituan/versions/afo-clients-0.5.70/afo-tensorflow/afo-tensorflow-0.5.70 /opt/meituan/tensorflow-release 试一下
println("Start!!!")

Start!!!


In [2]:
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
   new SparkConf()
   .setAppName(s"Online-Feature-For-Bid-${user}")
   .setMaster("yarn")
   .set("spark.yarn.queue", "root.zw03.hadoop-waimaiadrd.etl") // spark 的队列，可以根据自己的需要设置
   .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
   .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
   .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.sql.hive.metastorePartitionPruning", "true")
   .set("spark.shuffle.service.enabled", "true")
   .set("spark.dynamicAllocation.enabled", "true")
   .set("spark.dynamicAllocation.minExecutors", "600") //spark的执行Executors
   .set("spark.dynamicAllocation.maxExecutors","1500")
   .set("spark.executor.memory","16G")
   .set("spark.driver.memory","7G")
   .set("spark.input.dir.recursive","true")
   .set("spark.sql.ignore.existed.function.enable","true")
   .set("spark.yarn.maxAppAttempts", "2")
   .set("spark.noResourceMaxInterval","86400s")
   .set("spark.scheduler.mode", "FAIR")
   .set("spark.executor.cores","8")
   .set("spark.yarn.executor.memoryOverhead","32024")
   .set("spark.yarn.driver.memoryOverhead","32024")
   .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
 NotebookSparkSession
   .builder()
   .config(conf)
   .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
spark.sparkContext.applicationId
var logo="AFO_RL"
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
import java.text.SimpleDateFormat
import java.util.Date

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions prior to 1.8.
SLF4J: Ignoring binding found at [jar:file:/conda/envs/notebook/share/jupyter/kernels/scala211/pack/lib/slf4j-nop-1.7.26.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#ignoredBindings for an explanation.


Adding Hadoop conf dir /opt/meituan/hadoop/etc/hadoop to classpath
Adding Hive conf dir /opt/meituan/spark-2.2/conf to classpath
Creating SparkSession


log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Spark UI

import $ivy.$                            

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.{SparkConf, SparkContext}

user: String = "wanghao251"
conf: SparkConf = org.apache.spark.SparkConf@2873c3bf
spark: SparkSession = org.apache.spark.sql.SparkSession@4f39a420
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

res1_15: String = "application_1671244879475_22456637"
logo: String = "AFO_RL"
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

import java.text.SimpleDateFormat

import java.util.Date

In [3]:
// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    printCurTime()
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
    printCurTime()
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    printCurTime()
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    printCurTime()
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }



def printCurTime():Unit={
    println(new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date))
}


def showTopNFromSQL(sql:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,sql,true)
    result.show(n)
    printCurTime()
}
def executeSQLLazy(sql:String):DataFrame={
    executeSQL(spark.sqlContext,sql,true)
}
def showTopNFromTableName(tableName:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,s"""select * from ${tableName} limit ${n}""",true)
    result.show(n)
    printCurTime()
}
def executeShell(shell:String):Unit={
    printCurTime()
    println("----------------The content of shell is: -------------------")
    println(shell)
    println("----------------Start execution....-------------------")
    var file_name=System.currentTimeMillis()
    val writer = new PrintWriter(new File(s"/opt/meituan/dolphinfs_${user}/${file_name}.sh" ))
    writer.write(shell)
    writer.close()
    //启动脚本
    s"chmod +777 /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"sh /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"rm -r /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    println("----------------End-------------------")
    printCurTime()
}

def getFeatureMap(x:((String,String))):String={
    var x_1 = x._1
    var x_2 = x._2.replace("Type","")
    if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
    }
    var x_3 = s"""${x_1} ${x_2}"""
    x_3
}
// 传入sql，返回结果表
def createTableFromSQLWithName(sql:String,tableName:String,database:String):Unit={
    var rowData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var finalTableName=database+"."+tableName
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    
    
    //var featureForCreateTable=table_scheme.map{x=> s"""${x._1} ${x._2.stripSuffix("Type")}""".replace("Type","")}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    executeSQLLazy(s"insert OVERWRITE table ${finalTableName} select ${columnsName} from ${tmpTableName}")
}

def createTableFromSQL(sql:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,"tmp")
    "tmp."+finalTableName
}
def createTableFromSQL(sql:String,database:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,database)
    database+"."+finalTableName
}

def generateTrajectory(tableName:String, groupColumns:Array[String], rankColumns:Array[String], joinColumns:Array[String],database:String):String={
    
    println(tableName.split("."))
    var finalTableName=database+"."+tableName.split("\\.")(1)+"_trajectory"
    
    var table_scheme = executeSQLLazy(s"""select * from ${tableName} limit 1""").dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var groupColumnString=groupColumns.mkString(",")
    var rankColumnString = rankColumns.mkString(",")
    var joinConditionString = joinColumns.map(x=> s"t1.${x}=t2.${x} ").mkString("\n and ")
    var selectT1columnsNameString = table_scheme.map{x=> s"t1.${x._1}"}.mkString(",")
    var selectT2columnsNameString = table_scheme.map{x=> s"t2.${x._1} as next_${x._1}"}.mkString(",")
    //var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",") + ",rn int,"+table_scheme.map(x=> {
        var x_1 = x._1
        var x_2 = x._2.replace("Type","")
        if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
        }
        
        s"""next_${x_1} ${x_2}"""}
                                              ).mkString(",")+",next_rn int"
    var allColumnString=table_scheme.map{x=> s"${x._1}"}.mkString(",")+",rn,"+table_scheme.map{x=> s"next_${x._1}"}.mkString(",")+",next_rn"
    var sql=s"""
        select ${selectT1columnsNameString},t1.rn,${selectT2columnsNameString},t2.rn as next_rn
        from(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t1
        left join(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t2
        on t1.rn+1 = t2.rn
        and ${joinConditionString}
        """
    var trajectoryData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    trajectoryData.createOrReplaceTempView(tmpTableName)
    spark.sql(s"insert OVERWRITE table ${finalTableName} select ${allColumnString} from ${tmpTableName}")
    finalTableName
}
def createPTTableFromTable(tableName:String,targetTableName:String,database:String):String={
    var finalTableName=targetTableName
    var rowData=executeSQLLazy(s"select * from ${tableName} limit 1")
    var tmpTableName=s"${database}.${logo}_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt='${date_s}') select ${columnsName} from ${tmpTableName}")
    finalTableName
}

def createPTTableFromTableWithSpace(tableName:String,database:String):String={
    var finalTableName=database+s".${logo}_${System.currentTimeMillis()}"
    var rowData=executeSQLLazy(s"select * from ${tableName}")
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt=${date_s}) select ${columnsName} from ${tmpTableName}")
    finalTableName
}

import scala.collection.mutable.ArrayBuffer

import scala.math

import scala.util.Random;

import java.lang.Math

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.spark.sql.{DataFrame, Row}

import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}


import scala.collection.immutable.Map

import scala.collection.mutable


defined function JSONObj2Map
import org.apache.hadoop.hive.ql.exec.UDF

import spark.implicits._

findIndex2: String => String = <function1>
res2_13: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
res2_14: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
defined function getDate
defined function getTimeSuffix
defined function dropTable
defined function executeSQL
defined function genDateList
defined function printCurTime
defined function showTopNFromSQL
def

In [4]:
// 配置区
val is_execute_sql = true
var table_suffix = "20230118"
var befor1day = "20230118"
var fp_log_date_start="20230111"
var fp_log_date_end="20230118"
var predict_dt="20230117"
var business="dsa_os_unify_model" //提取特征的业务线

is_execute_sql: Boolean = true
table_suffix: String = "20230118"
befor1day: String = "20230118"
fp_log_date_start: String = "20230111"
fp_log_date_end: String = "20230118"
predict_dt: String = "20230117"
business: String = "dsa_os_unify_model"

In [5]:
// 获取featureID 和名字的对应关系
var featureId2Name=s"""
select distinct id, name
from origindb_ss.waimaiadtcpc_waimai_offline_feature_platform__wm_model_feature_map
where dt=${befor1day}
and status=0
"""
//构建 id到name映射的map
val featureId2NameMap = spark.sqlContext.sql(featureId2Name).na.drop().rdd.
      map(f => (f.getLong(0).toString, f.getString(1))).
      collectAsMap()


val feaArray = Array(
    "pvid string",
    "user_id bigint",
    "uuid string",
    "poi_id bigint",
    "request_time bigint",
    "precisebid double",
    "demandbiztype bigint",
    "supplybiztype bigint",
    "is_expose bigint",
    "is_click bigint",
    "final_charge double",
    "is_order bigint",
    "revenue double",
    "rt_features string",
    "raw_features string",
    "modelname string",
    "business string",
    "params string",
    "dt string"
)

var logRawFeature=""

// 构建schema
import scala.collection.mutable.ArrayBuffer
val schemaArray = new ArrayBuffer[StructField]()
var columeNameSet= new mutable.HashSet[String]()
for (kv <- feaArray.map(_.split(" "))) {
    val groupKey = kv.head
    val groupType = kv.last
    columeNameSet.add(groupKey)
    logRawFeature+=groupKey+","
    if (groupType.startsWith("double")) {
        schemaArray += StructField(groupKey, DoubleType)
    } else if (groupType.startsWith("int")) {
        schemaArray += StructField(groupKey, IntegerType)
    } else if (groupType.startsWith("bigint")) {
        schemaArray += StructField(groupKey, LongType)
    } else {
        schemaArray += StructField(groupKey, StringType)
    }
}
logRawFeature=logRawFeature.stripSuffix(",")


var line=executeSQLLazy(s"""
select rt_features,raw_features,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt = "${fp_log_date_end}"
limit 1
""")
//scame 构建
var rt_map:Map[String, String] = Map()
var rawFeatureMap:Map[String, String] = Map()



line.collect().foreach(x=>{
    var rt_features=x.getString(0)
    var rawFeatureString= x.getString(1)
    var detailsString=x.getString(2)
    println("before")
    println(rt_features)
    println(rawFeatureString)
    var rtMap =  JSONObj2Map(JSON.parseObject(rt_features))
    var rawFeature =  JSONObj2Map(JSON.parseObject(rawFeatureString))
    var detail=JSONObj2Map(JSON.parseObject(detailsString)).get("preciseBidFinalDetailMap").get
    println(detail)
    for(key<-rtMap.keys){
        //类型增加判断 todo:
        if(!columeNameSet.contains(featureId2NameMap.getOrElse(key, key))){
            //println(s"${key}:${rtMap.get(key).get}, ${rtMap.get(key).get.getClass.getSimpleName}")
            schemaArray+= StructField(featureId2NameMap.getOrElse(key, key), StringType)
            rt_map+=(key->featureId2NameMap.getOrElse(key, key))
            columeNameSet.add(featureId2NameMap.getOrElse(key, key))
        }
    }
    for(key<-rawFeature.keys){
        if(!columeNameSet.contains(key)){
            // println(s"${key}:${rawFeature.get(key).get}, ${rawFeature.get(key).get.getClass.getSimpleName}")
            schemaArray+= StructField(key, StringType)
            rawFeatureMap+=(key->key)
            columeNameSet.add(featureId2NameMap.getOrElse(key, key))
        }
    }
    
})

var detail_feature_list = Array("rlkAct", "delete_res")
for (key <-detail_feature_list){
    schemaArray+= StructField(key, StringType)
    columeNameSet.add(key)
}
println("after")
println(rt_map)
println(rawFeatureMap)

rdd at cmd4.sc:8

1 / 1

collectAsMap at cmd4.sc:10

200 / 200

2023-01-19 10:23:06
执行 
 
select rt_features,raw_features,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt = "20230118"
limit 1

2023-01-19 10:23:06


collect at cmd4.sc:71

1 / 1

before
{"92310":4.0,"68503":0.0,"146102":0.0,"8531":10.0,"286":0.0}
{"real_cpc_push":0.0,"real_impr_bj":2.0,"upperk":1.2,"charge_before1week":0.0,"real_charge_push":0.0,"real_budget":5000.0,"impr_before3week":74.0,"real_order_push":0.0,"lowerk":0.9,"real_impr_push":3.0,"impr":1.0,"real_csm":729.0,"real_cpc_ss":0.0,"real_impr_ss":3.0,"ctr":0.06994468800723554,"click_before2week":1.0,"revenue_before1week":0.0,"real_cpc_bj":0.0,"mt_butie_before1week":0.0,"real_revenue_push":0.0,"impr_before1week":3.0,"cityid":441702.0,"mt_butie":0.0,"feacomp_ctr":1.5990733762113312,"revenue_before3week":80.4000015258789,"order_num_before2week":0.0,"cpc_before4week":0.0,"real_charge_bj":0.0,"ratioctr":1.9341313537353604,"ratio_aimcpc":0.8770109850672618,"mt_butie_before3week":0.0,"real_click":0.0,"click_before3week":3.0,"real_order_bj":0.0,"charge_before4week":0.0,"real_revenue":0.0,"cpc_before2week":0.18000000715255737,"real_click_dj":0.0,"prod":1.0,"charge":0.0,"real_charge_ss":0.0,"charge_before2week":0

featureId2Name: String = """
select distinct id, name
from origindb_ss.waimaiadtcpc_waimai_offline_feature_platform__wm_model_feature_map
where dt=20230118
and status=0
"""
featureId2NameMap: collection.Map[String, String] = Map(
  "152664" -> "thh_main_hot_gmv_poi_order_cnt_3day",
  "68746" -> "sg_b2c_product_comment_cnt_30day",
  "128635" -> "thh_whole_scene_uuid_prod_second_cate_order_cnt_90day",
  "8946" -> "query_poi_max_query_term_poi_vooeo_7d",
  "16136" -> "poi_franchise_poislot_30_90_pv",
  "146004" -> "sg_search_stg_user_view_poi_count_30d",
  "68440" -> "sg_search_query_spuname_bert_semantic_relevance_prob",
  "191468" -> "cpc_ad_nl_req_avg_ctr",
  "165028" -> "thh_poi_all_scene_view_cnt_3day",
  "93546" -> "sg_search_rt_spu_all_discount_amount_l1_origin",
  "10481" -> "rel_food_count",
  "76054" -> "query_click_top20_poi_list_90day",
  "590" -> "ka_order_90days",
  "77787" -> "hoae_cvr_tagid_order_count_30_v1",
  "10667" -> "list_click_poi_embedding_cur",
  "157435" -> "log

In [6]:
//将log表的特征列展开
var tmpFPExtendRTFeatureTableName=s"${user}_${business}_${System.currentTimeMillis()}"
var FPExtendRTFeaturetableName=s"tmp.dsa_model_fp_log_${user}_${business}_df_${fp_log_date_end}"

var logData=s"""
select ${logRawFeature},biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt between ${fp_log_date_start} and ${fp_log_date_end} and is_expose>0
and business ="${business}"
"""
println(logData)

var rawData = spark.sqlContext.sql(logData).rdd.flatMap{
      f =>
        var pvid=f.getAs[String]("pvid")
        var user_id=f.getAs[Long]("user_id")
        var uuid=f.getAs[String]("uuid")
        var poi_id=f.getAs[Long]("poi_id")
        var request_time=f.getAs[Long]("request_time")
        var precisebid=f.getAs[Double]("precisebid")
        var demandbiztype=f.getAs[Long]("demandbiztype")
        var supplybiztype=f.getAs[Long]("supplybiztype")
        var is_expose=f.getAs[Long]("is_expose")
        var is_click=f.getAs[Long]("is_click")
        var final_charge=f.getAs[Double]("final_charge")
        var is_order=f.getAs[Long]("is_order")
        var revenue=f.getAs[Double]("revenue")
        var rt_features=f.getAs[String]("rt_features").replace("Infinity","-1.0")
        var raw_features=f.getAs[String]("raw_features").replace("Infinity","-1.0")
        var modelname=f.getAs[String]("modelname")
        var business=f.getAs[String]("business")
        var params=f.getAs[String]("params")
        var dt = f.getAs[String]("dt")
        var detail_features =f.getAs[String]("biddetails").replace("Infinity","-1.0")
        var array =Array(pvid,user_id,uuid,poi_id,request_time,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt)
        var rtFeaturesMap = JSONObj2Map(JSON.parseObject(rt_features))
        var rawFeaturesLawMap = JSONObj2Map(JSON.parseObject(raw_features))
        var preciseBidFinalDetailMap = JSONObj2Map(JSON.parseObject(detail_features)).getOrElse("preciseBidFinalDetailMap","{}")
        var detail=JSONObj2Map(JSON.parseObject(preciseBidFinalDetailMap))
        //当前只支持int，string和double的类型，复杂类型还未支持
        for(featureID<-rt_map.keys){
            if(rtFeaturesMap.contains(featureID)){
                array=array:+rtFeaturesMap.get(featureID).get
            }else{
                array=array:+ -0.678
            }
        }
        for(featureID<-rawFeatureMap.keys){
            if(rawFeatureMap.contains(featureID)){
                array=array:+rawFeaturesLawMap.get(featureID).get
            }else{
                array=array:+"unknown"
            }
        }
        
        for(featureID<-detail_feature_list){
            if(detail.contains(featureID)){
                array=array:+detail.get(featureID).get
            }else{
                array=array:+ "unknown"
            }
        }
        
        var row =Row(array:_*)
        // var row = Row(pvid)
        List(row)
}
println(rawData.count())
println("test")


select pvid,user_id,uuid,poi_id,request_time,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt between 20230111 and 20230118 and is_expose>0
and business ="dsa_os_unify_model"



count at cmd5.sc:67

16008 / 16000

567875781
test


tmpFPExtendRTFeatureTableName: String = "wanghao251_dsa_os_unify_model_1674094991345"
FPExtendRTFeaturetableName: String = "tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230118"
logData: String = """
select pvid,user_id,uuid,poi_id,request_time,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt,biddetails
from mart_waimaiad.dsa_model_fp_os_rl_log_wh
where dt between 20230111 and 20230118 and is_expose>0
and business ="dsa_os_unify_model"
"""
rawData: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[47] at flatMap at cmd5.sc:12

In [7]:
// 将数据写入到hive表
val tmpTable = spark.sqlContext.createDataFrame(rawData, StructType(schemaArray.toList))
tmpTable.toDF().registerTempTable(tmpFPExtendRTFeatureTableName)
var featureForCreateTable=s""
var allFeatures=""
for(i <- 0 to schemaArray.length-1){
    allFeatures=allFeatures+schemaArray(i).name+","
    if(i==schemaArray.length-1){
        featureForCreateTable=featureForCreateTable+schemaArray(i).name+" "+schemaArray(i).dataType.typeName
    }else{
        featureForCreateTable=featureForCreateTable+schemaArray(i).name+" "+schemaArray(i).dataType.typeName+","
    }
}
allFeatures=allFeatures.stripSuffix(",")
executeSQLLazy(s"""drop table if exists $FPExtendRTFeaturetableName""")
executeSQLLazy(s"""create table if not exists $FPExtendRTFeaturetableName(${featureForCreateTable}) STORED AS ORC""")
executeSQLLazy(s"""insert overwrite table ${FPExtendRTFeaturetableName} select ${allFeatures} from ${tmpFPExtendRTFeatureTableName}
where cast(rawbid as float)+cast(precisebid as float)+cast(final_charge as float)+cast(click as float)+cast(user_tag_pref_click_3day as float)+cast(hour as float)+cast(user_ord_num_weekdays as float)+cast(charge_before2week as float)+cast(order_num_before2week as float)+cast(impr_before1week as float)+cast(feacomp_cvr as float)+cast(click_before1week as float)+cast(lowerk as float)+cast(real_cpc as float)+cast(real_csm as float)+cast(revenue_before3week as float)+cast(cpc_before1week as float)+cast(charge_before3week as float)+cast(winningscore_comp12 as float)+cast(impr as float)+cast(real_cst_bdgt as float)+cast(real_aBln as float)+cast(feacomp_ctr as float)+cast(real_expose as float)+cast(real_revenue as float)+cast(revenue_before2week as float)+cast(ratioctr as float)+cast(real_price as float)+cast(lowerbid as float)+cast(ratio_aimcpc as float)+cast(cpc_before4week as float)+cast(order_num_before3week as float)+cast(histctr as float)+cast(mt_butie_before2week as float)+cast(mt_butie_before3week as float)+cast(real_budget as float)+cast(order_num_before4week as float)+cast(cpc as float)+cast(order_num_before1week as float)+cast(revenue_before1week as float)+cast(mt_butie as float)+cast(cpc_before2week as float)+cast(cpc_before3week as float)+cast(aimcpc as float)+cast(mt_butie_before1week as float)+cast(real_csm_bln as float)+cast(click_before3week as float)+cast(charge_before1week as float)+cast(click_before4week as float)+cast(click_before2week as float)+cast(real_click as float)+cast(winningscore_qs as float)+cast(cityid as float)+cast(ratio_click as float)+cast(prod as float)+cast(ratioctr_exp as float)+cast(upperk as float)+cast(charge as float)+cast(revenue_before4week as float)+cast(ctr as float)+cast(order_num as float)+cast(impr_before4week as float)+cast(impr_before2week as float)+cast(mt_butie_before4week as float)+cast(real_order as float)+cast(ratio_cpc as float)+cast(impr_before3week as float)+cast(charge_before4week as float)+cast(real_bln as float) < 1E6""")
// showTopNFromTableName(tmpFPExtendRTFeatureTableName,10)

2023-01-19 10:27:41
执行 
 drop table if exists tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230118
2023-01-19 10:27:41
2023-01-19 10:27:41
执行 
 create table if not exists tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230118(pvid string,user_id long,uuid string,poi_id long,request_time long,precisebid double,demandbiztype long,supplybiztype long,is_expose long,is_click long,final_charge double,is_order long,revenue double,rt_features string,raw_features string,modelname string,business string,params string,dt string,unused string,click string,user_tag_pref_click_3day string,hour string,user_ord_num_weekdays string,charge_before2week string,order_num_before2week string,impr_before1week string,feacomp_cvr string,real_charge_push string,click_before1week string,lowerk string,real_click_push string,real_cpc string,real_csm string,real_order_bj string,revenue_before3week string,cpc_before1week string,charge_before3week string,winningscore_comp12 string,impr string,real_ch

sql at cmd2.sc:66

16016 / 16000

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230118' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2023-01-19 10:32:10


tmpTable: DataFrame = [pvid: string, user_id: bigint ... 116 more fields]
featureForCreateTable: String = "pvid string,user_id long,uuid string,poi_id long,request_time long,precisebid double,demandbiztype long,supplybiztype long,is_expose long,is_click long,final_charge double,is_order long,revenue double,rt_features string,raw_features string,modelname string,business string,params string,dt string,unused string,click string,user_tag_pref_click_3day string,hour string,user_ord_num_weekdays string,charge_before2week string,order_num_before2week string,impr_before1week string,feacomp_cvr string,real_charge_push string,click_before1week string,lowerk string,real_click_push string,real_cpc string,real_csm string,real_order_bj string,revenue_before3week string,cpc_before1week string,charge_before3week string,winningscore_comp12 string,impr string,real_charge_dj string,real_order_push string,real_cst_bdgt string,real_aBln string,feacomp_ctr string,real_expose string,real_impr_push string,r

In [8]:
//需要修改的地方：根据生产的大宽表进行数据加工
// ------------------------------------------
// ------------------------------------------
// 多智能体的环境应该是，当前时间点，四个渠道的obs拼接
var processTable = createTableFromSQL(s"""
SELECT dt,
       pvid,
       user_id,
       uuid,
       poi_id,
       request_time,
       prod,
       rlkAct,
       rawbid_dj + rawbid_bj + rawbid_ss + rawbid_push as rawbid_single,
       precisebid_dj + precisebid_bj + precisebid_ss + precisebid_push as precisebid_single,
       array(user_tag_pref_click_3day_dj, user_tag_pref_click_3day_bj, user_tag_pref_click_3day_ss, user_tag_pref_click_3day_push) as user_tag_pref_click_3day,
       array(rawbid_dj, rawbid_bj, rawbid_ss, rawbid_push) as rawbid,
       array(precisebid_dj, precisebid_bj, precisebid_ss, precisebid_push) as precisebid,
       array(supplybiztype_dj, supplybiztype_bj, supplybiztype_ss, supplybiztype_push) as supplybiztype,
       array(is_expose_dj, is_expose_bj, is_expose_ss, is_expose_push) as is_expose,
       array(is_click_dj, is_click_bj, is_click_ss, is_click_push) as is_click,
       array(final_charge_dj, final_charge_bj, final_charge_ss, final_charge_push) as final_charge,
       array(is_order_dj, is_order_bj, is_order_ss, is_order_push) as is_order,
       array(revenue_dj, revenue_bj, revenue_ss, revenue_push) as revenue,
       array(unused_dj, unused_bj, unused_ss, unused_push) as unused,
       array(click_dj, click_bj, click_ss, click_push) as click,
       array(hour_dj, hour_bj, hour_ss, hour_push) as hour,
       array(user_ord_num_weekdays_dj, user_ord_num_weekdays_bj, user_ord_num_weekdays_ss, user_ord_num_weekdays_push) as user_ord_num_weekdays,
       array(charge_before2week_dj, charge_before2week_bj, charge_before2week_ss, charge_before2week_push) as charge_before2week,
       array(order_num_before2week_dj, order_num_before2week_bj, order_num_before2week_ss, order_num_before2week_push) as order_num_before2week,
       array(impr_before1week_dj, impr_before1week_bj, impr_before1week_ss, impr_before1week_push) as impr_before1week,
       array(feacomp_cvr_dj, feacomp_cvr_bj, feacomp_cvr_ss, feacomp_cvr_push) as feacomp_cvr,
       array(click_before1week_dj, click_before1week_bj, click_before1week_ss, click_before1week_push) as click_before1week,
       array(lowerk_dj, lowerk_bj, lowerk_ss, lowerk_push) as lowerk,

       array(revenue_before3week_dj, revenue_before3week_bj, revenue_before3week_ss, revenue_before3week_push) as revenue_before3week,
       array(cpc_before1week_dj, cpc_before1week_bj, cpc_before1week_ss, cpc_before1week_push) as cpc_before1week,
       array(charge_before3week_dj, charge_before3week_bj, charge_before3week_ss, charge_before3week_push) as charge_before3week,
       array(winningscore_comp12_dj, winningscore_comp12_bj, winningscore_comp12_ss, winningscore_comp12_push) as winningscore_comp12,
       array(impr_dj, impr_bj, impr_ss, impr_push) as impr,
       array(real_cst_bdgt_dj, real_cst_bdgt_bj, real_cst_bdgt_ss, real_cst_bdgt_push) as real_cst_bdgt,
       array(real_aBln_dj, real_aBln_bj, real_aBln_ss, real_aBln_push) as real_aBln,
       array(feacomp_ctr_dj, feacomp_ctr_bj, feacomp_ctr_ss, feacomp_ctr_push) as feacomp_ctr,
       
       array(real_impr_dj, real_impr_bj, real_impr_ss, real_impr_push) as real_expose,
       array(real_click_dj, real_click_bj, real_click_ss, real_click_push) as real_click,
       array(real_order_dj, real_order_bj, real_order_ss, real_order_push) as real_order,
       array(real_cpc_dj, real_cpc_bj, real_cpc_ss, real_cpc_push) as real_cpc,
       array(real_charge_dj, real_charge_bj, real_charge_ss, real_charge_push) as real_csm,
       array(real_cash_dj, real_cash_bj, real_cash_ss, real_cash_push) as real_cash,
       
         array(real_expose_all_dj, real_expose_all_bj, real_expose_all_ss, real_expose_all_push) as real_expose_all,
        array(real_click_all_dj, real_click_all_bj, real_click_all_ss, real_click_all_push) as real_click_all,
        array(real_order_all_dj, real_order_all_bj, real_order_all_ss, real_order_all_push) as real_order_all,
        array(real_cpc_all_dj, real_cpc_all_bj, real_cpc_all_ss, real_cpc_all_push) as real_cpc_all,
        array(real_csm_all_dj, real_csm_all_bj, real_csm_all_ss, real_csm_all_push) as real_csm_all,
        array(real_revenue_all_dj, real_revenue_all_bj, real_revenue_all_ss, real_revenue_all_push) as real_revenue_all,
       
    
       array(real_revenue_dj, real_revenue_bj, real_revenue_ss, real_revenue_push) as real_revenue,
       array(revenue_before2week_dj, revenue_before2week_bj, revenue_before2week_ss, revenue_before2week_push) as revenue_before2week,
       array(ratioctr_dj, ratioctr_bj, ratioctr_ss, ratioctr_push) as ratioctr,
       array(real_price_dj, real_price_bj, real_price_ss, real_price_push) as real_price,
       array(lowerbid_dj, lowerbid_bj, lowerbid_ss, lowerbid_push) as lowerbid,
       array(ratio_aimcpc_dj, ratio_aimcpc_bj, ratio_aimcpc_ss, ratio_aimcpc_push) as ratio_aimcpc,
       array(cpc_before4week_dj, cpc_before4week_bj, cpc_before4week_ss, cpc_before4week_push) as cpc_before4week,
       array(order_num_before3week_dj, order_num_before3week_bj, order_num_before3week_ss, order_num_before3week_push) as order_num_before3week,
       array(histctr_dj, histctr_bj, histctr_ss, histctr_push) as histctr,
       array(mt_butie_before2week_dj, mt_butie_before2week_bj, mt_butie_before2week_ss, mt_butie_before2week_push) as mt_butie_before2week,
       array(mt_butie_before3week_dj, mt_butie_before3week_bj, mt_butie_before3week_ss, mt_butie_before3week_push) as mt_butie_before3week,
       array(real_budget_dj, real_budget_bj, real_budget_ss, real_budget_push) as real_budget,
       array(order_num_before4week_dj, order_num_before4week_bj, order_num_before4week_ss, order_num_before4week_push) as order_num_before4week,
       array(cpc_dj, cpc_bj, cpc_ss, cpc_push) as cpc,
       array(order_num_before1week_dj, order_num_before1week_bj, order_num_before1week_ss, order_num_before1week_push) as order_num_before1week,
       array(revenue_before1week_dj, revenue_before1week_bj, revenue_before1week_ss, revenue_before1week_push) as revenue_before1week,
       array(mt_butie_dj, mt_butie_bj, mt_butie_ss, mt_butie_push) as mt_butie,
       array(cpc_before2week_dj, cpc_before2week_bj, cpc_before2week_ss, cpc_before2week_push) as cpc_before2week,
       array(cpc_before3week_dj, cpc_before3week_bj, cpc_before3week_ss, cpc_before3week_push) as cpc_before3week,
       array(aimcpc_dj, aimcpc_bj, aimcpc_ss, aimcpc_push) as aimcpc,
       array(mt_butie_before1week_dj, mt_butie_before1week_bj, mt_butie_before1week_ss, mt_butie_before1week_push) as mt_butie_before1week,
       array(real_csm_bln_dj, real_csm_bln_bj, real_csm_bln_ss, real_csm_bln_push) as real_csm_bln,
       array(click_before3week_dj, click_before3week_bj, click_before3week_ss, click_before3week_push) as click_before3week,
       array(charge_before1week_dj, charge_before1week_bj, charge_before1week_ss, charge_before1week_push) as charge_before1week,
       array(click_before4week_dj, click_before4week_bj, click_before4week_ss, click_before4week_push) as click_before4week,
       array(click_before2week_dj, click_before2week_bj, click_before2week_ss, click_before2week_push) as click_before2week,
       array(winningscore_qs_dj, winningscore_qs_bj, winningscore_qs_ss, winningscore_qs_push) as winningscore_qs,
       array(cityid_dj, cityid_bj, cityid_ss, cityid_push) as cityid,
       array(ratio_click_dj, ratio_click_bj, ratio_click_ss, ratio_click_push) as ratio_click,
       array(ratioctr_exp_dj, ratioctr_exp_bj, ratioctr_exp_ss, ratioctr_exp_push) as ratioctr_exp,
       array(upperk_dj, upperk_bj, upperk_ss, upperk_push) as upperk,
       array(charge_dj, charge_bj, charge_ss, charge_push) as charge,
       array(revenue_before4week_dj, revenue_before4week_bj, revenue_before4week_ss, revenue_before4week_push) as revenue_before4week,
       array(ctr_dj, ctr_bj, ctr_ss, ctr_push) as ctr,
       array(order_num_dj, order_num_bj, order_num_ss, order_num_push) as order_num,
       array(impr_before4week_dj, impr_before4week_bj, impr_before4week_ss, impr_before4week_push) as impr_before4week,
       array(impr_before2week_dj, impr_before2week_bj, impr_before2week_ss, impr_before2week_push) as impr_before2week,
       array(mt_butie_before4week_dj, mt_butie_before4week_bj, mt_butie_before4week_ss, mt_butie_before4week_push) as mt_butie_before4week,
       array(ratio_cpc_dj, ratio_cpc_bj, ratio_cpc_ss, ratio_cpc_push) as ratio_cpc,
       array(impr_before3week_dj, impr_before3week_bj, impr_before3week_ss, impr_before3week_push) as impr_before3week,
       array(charge_before4week_dj, charge_before4week_bj, charge_before4week_ss, charge_before4week_push) as charge_before4week,
       array(real_bln_dj, real_bln_bj, real_bln_ss, real_bln_push) as real_bln
  FROM (
        SELECT dt,
               pvid,
               user_id,
               uuid,
               poi_id,
               request_time,
               prod,
               if(prod=1.0, final_charge, 0) as jifei_dj,
               if(prod=1.0, user_tag_pref_click_3day, 0) as user_tag_pref_click_3day_dj,
               if(prod=1.0, rawbid, 0) as rawbid_dj,
               if(prod=1.0, precisebid, 0) as precisebid_dj,
               if(prod=1.0, supplybiztype, 0) as supplybiztype_dj,
               if(prod=1.0, is_expose, 0) as is_expose_dj,
               if(prod=1.0, is_click, 0) as is_click_dj,
               if(prod=1.0, final_charge, 0) as final_charge_dj,
               if(prod=1.0, is_order, 0) as is_order_dj,
               if(prod=1.0, revenue, 0) as revenue_dj,
               if(prod=1.0, unused, 0) as unused_dj,
               if(prod=1.0, click, 0) as click_dj,     
               if(prod=1.0, hour, 0) as hour_dj,
               if(prod=1.0, user_ord_num_weekdays, 0) as user_ord_num_weekdays_dj,
               if(prod=1.0, charge_before2week, 0) as charge_before2week_dj,
               if(prod=1.0, order_num_before2week, 0) as order_num_before2week_dj,
               if(prod=1.0, impr_before1week, 0) as impr_before1week_dj,
               if(prod=1.0, feacomp_cvr, 0) as feacomp_cvr_dj,
               if(prod=1.0, click_before1week, 0) as click_before1week_dj,
               if(prod=1.0, lowerk, 0) as lowerk_dj,
               if(prod=1.0, revenue_before3week, 0) as revenue_before3week_dj,
               if(prod=1.0, cpc_before1week, 0) as cpc_before1week_dj,
               if(prod=1.0, charge_before3week, 0) as charge_before3week_dj,
               if(prod=1.0, winningscore_comp12, 0) as winningscore_comp12_dj,
               if(prod=1.0, impr, 0) as impr_dj,
               if(prod=1.0, real_cst_bdgt, 0) as real_cst_bdgt_dj,
               if(prod=1.0, real_aBln, 0) as real_aBln_dj,
               if(prod=1.0, feacomp_ctr, 0) as feacomp_ctr_dj,
               if(prod=1.0, revenue_before2week, 0) as revenue_before2week_dj,
               if(prod=1.0, ratioctr, 0) as ratioctr_dj,
               if(prod=1.0, real_price, 0) as real_price_dj,
               if(prod=1.0, lowerbid, 0) as lowerbid_dj,
               if(prod=1.0, ratio_aimcpc, 0) as ratio_aimcpc_dj,
               if(prod=1.0, cpc_before4week, 0) as cpc_before4week_dj,
               if(prod=1.0, order_num_before3week, 0) as order_num_before3week_dj,
               if(prod=1.0, histctr, 0) as histctr_dj,
               if(prod=1.0, mt_butie_before2week, 0) as mt_butie_before2week_dj,
               if(prod=1.0, mt_butie_before3week, 0) as mt_butie_before3week_dj,
               if(prod=1.0, real_budget, 0) as real_budget_dj,
               if(prod=1.0, order_num_before4week, 0) as order_num_before4week_dj,
               if(prod=1.0, cpc, 0) as cpc_dj,
               if(prod=1.0, order_num_before1week, 0) as order_num_before1week_dj,
               if(prod=1.0, revenue_before1week, 0) as revenue_before1week_dj,
               if(prod=1.0, mt_butie, 0) as mt_butie_dj,
               if(prod=1.0, cpc_before2week, 0) as cpc_before2week_dj,
               if(prod=1.0, cpc_before3week, 0) as cpc_before3week_dj,
               if(prod=1.0, aimcpc, 0) as aimcpc_dj,
               if(prod=1.0, mt_butie_before1week, 0) as mt_butie_before1week_dj,
               if(prod=1.0, real_csm_bln, 0) as real_csm_bln_dj,
               if(prod=1.0, click_before3week, 0) as click_before3week_dj,
               if(prod=1.0, charge_before1week, 0) as charge_before1week_dj,
               if(prod=1.0, click_before4week, 0) as click_before4week_dj,
               if(prod=1.0, click_before2week, 0) as click_before2week_dj,
               if(prod=1.0, winningscore_qs, 0) as winningscore_qs_dj,
               if(prod=1.0, cityid, 0) as cityid_dj,
               if(prod=1.0, ratio_click, 0) as ratio_click_dj,
               if(prod=1.0, ratioctr_exp, 0) as ratioctr_exp_dj,
               if(prod=1.0, upperk, 0) as upperk_dj,
               if(prod=1.0, charge, 0) as charge_dj,
               if(prod=1.0, revenue_before4week, 0) as revenue_before4week_dj,
               if(prod=1.0, ctr, 0) as ctr_dj,
               if(prod=1.0, order_num, 0) as order_num_dj,
               if(prod=1.0, impr_before4week, 0) as impr_before4week_dj,
               if(prod=1.0, impr_before2week, 0) as impr_before2week_dj,
               if(prod=1.0, mt_butie_before4week, 0) as mt_butie_before4week_dj,
               if(prod=1.0, ratio_cpc, 0) as ratio_cpc_dj,
               if(prod=1.0, impr_before3week, 0) as impr_before3week_dj,
               if(prod=1.0, charge_before4week, 0) as charge_before4week_dj,
               if(prod=1.0, real_bln, 0) as real_bln_dj,
               
               if(prod=2.0, user_tag_pref_click_3day, 0) as user_tag_pref_click_3day_bj,
               if(prod=2.0, rawbid, 0) as rawbid_bj,
               if(prod=2.0, precisebid, 0) as precisebid_bj,
               if(prod=2.0, supplybiztype, 0) as supplybiztype_bj,
               if(prod=2.0, is_expose, 0) as is_expose_bj,
               if(prod=2.0, is_click, 0) as is_click_bj,
               if(prod=2.0, final_charge, 0) as final_charge_bj,
               if(prod=2.0, is_order, 0) as is_order_bj,
               if(prod=2.0, revenue, 0) as revenue_bj,
               if(prod=2.0, unused, 0) as unused_bj,
               if(prod=2.0, click, 0) as click_bj,           
               if(prod=2.0, hour, 0) as hour_bj,
               if(prod=2.0, user_ord_num_weekdays, 0) as user_ord_num_weekdays_bj,
               if(prod=2.0, charge_before2week, 0) as charge_before2week_bj,
               if(prod=2.0, order_num_before2week, 0) as order_num_before2week_bj,
               if(prod=2.0, impr_before1week, 0) as impr_before1week_bj,
               if(prod=2.0, feacomp_cvr, 0) as feacomp_cvr_bj,
               if(prod=2.0, click_before1week, 0) as click_before1week_bj,
               if(prod=2.0, lowerk, 0) as lowerk_bj,
               if(prod=2.0, revenue_before3week, 0) as revenue_before3week_bj,
               if(prod=2.0, cpc_before1week, 0) as cpc_before1week_bj,
               if(prod=2.0, charge_before3week, 0) as charge_before3week_bj,
               if(prod=2.0, winningscore_comp12, 0) as winningscore_comp12_bj,
               if(prod=2.0, impr, 0) as impr_bj,
               if(prod=2.0, real_cst_bdgt, 0) as real_cst_bdgt_bj,
               if(prod=2.0, real_aBln, 0) as real_aBln_bj,
               if(prod=2.0, feacomp_ctr, 0) as feacomp_ctr_bj,
               if(prod=2.0, revenue_before2week, 0) as revenue_before2week_bj,
               if(prod=2.0, ratioctr, 0) as ratioctr_bj,
               if(prod=2.0, real_price, 0) as real_price_bj,
               if(prod=2.0, lowerbid, 0) as lowerbid_bj,
               if(prod=2.0, ratio_aimcpc, 0) as ratio_aimcpc_bj,
               if(prod=2.0, cpc_before4week, 0) as cpc_before4week_bj,
               if(prod=2.0, order_num_before3week, 0) as order_num_before3week_bj,
               if(prod=2.0, histctr, 0) as histctr_bj,
               if(prod=2.0, mt_butie_before2week, 0) as mt_butie_before2week_bj,
               if(prod=2.0, mt_butie_before3week, 0) as mt_butie_before3week_bj,
               if(prod=2.0, real_budget, 0) as real_budget_bj,
               if(prod=2.0, order_num_before4week, 0) as order_num_before4week_bj,
               if(prod=2.0, cpc, 0) as cpc_bj,
               if(prod=2.0, order_num_before1week, 0) as order_num_before1week_bj,
               if(prod=2.0, revenue_before1week, 0) as revenue_before1week_bj,
               if(prod=2.0, mt_butie, 0) as mt_butie_bj,
               if(prod=2.0, cpc_before2week, 0) as cpc_before2week_bj,
               if(prod=2.0, cpc_before3week, 0) as cpc_before3week_bj,
               if(prod=2.0, aimcpc, 0) as aimcpc_bj,
               if(prod=2.0, mt_butie_before1week, 0) as mt_butie_before1week_bj,
               if(prod=2.0, real_csm_bln, 0) as real_csm_bln_bj,
               if(prod=2.0, click_before3week, 0) as click_before3week_bj,
               if(prod=2.0, charge_before1week, 0) as charge_before1week_bj,
               if(prod=2.0, click_before4week, 0) as click_before4week_bj,
               if(prod=2.0, click_before2week, 0) as click_before2week_bj,
               if(prod=2.0, winningscore_qs, 0) as winningscore_qs_bj,
               if(prod=2.0, cityid, 0) as cityid_bj,
               if(prod=2.0, ratio_click, 0) as ratio_click_bj,
               if(prod=2.0, ratioctr_exp, 0) as ratioctr_exp_bj,
               if(prod=2.0, upperk, 0) as upperk_bj,
               if(prod=2.0, charge, 0) as charge_bj,
               if(prod=2.0, revenue_before4week, 0) as revenue_before4week_bj,
               if(prod=2.0, ctr, 0) as ctr_bj,
               if(prod=2.0, order_num, 0) as order_num_bj,
               if(prod=2.0, impr_before4week, 0) as impr_before4week_bj,
               if(prod=2.0, impr_before2week, 0) as impr_before2week_bj,
               if(prod=2.0, mt_butie_before4week, 0) as mt_butie_before4week_bj,
               if(prod=2.0, ratio_cpc, 0) as ratio_cpc_bj,
               if(prod=2.0, impr_before3week, 0) as impr_before3week_bj,
               if(prod=2.0, charge_before4week, 0) as charge_before4week_bj,
               if(prod=2.0, real_bln, 0) as real_bln_bj,
               
               if(prod=3.0, user_tag_pref_click_3day, 0) as user_tag_pref_click_3day_ss,
               if(prod=3.0, rawbid, 0) as rawbid_ss,
               if(prod=3.0, precisebid, 0) as precisebid_ss,
               if(prod=3.0, supplybiztype, 0) as supplybiztype_ss,
               if(prod=3.0, is_expose, 0) as is_expose_ss,
               if(prod=3.0, is_click, 0) as is_click_ss,
               if(prod=3.0, final_charge, 0) as final_charge_ss,
               if(prod=3.0, is_order, 0) as is_order_ss,
               if(prod=3.0, revenue, 0) as revenue_ss,
               if(prod=3.0, unused, 0) as unused_ss,
               if(prod=3.0, click, 0) as click_ss,

               if(prod=3.0, hour, 0) as hour_ss,
               if(prod=3.0, user_ord_num_weekdays, 0) as user_ord_num_weekdays_ss,
               if(prod=3.0, charge_before2week, 0) as charge_before2week_ss,
               if(prod=3.0, order_num_before2week, 0) as order_num_before2week_ss,
               if(prod=3.0, impr_before1week, 0) as impr_before1week_ss,
               if(prod=3.0, feacomp_cvr, 0) as feacomp_cvr_ss,
               if(prod=3.0, click_before1week, 0) as click_before1week_ss,
               if(prod=3.0, lowerk, 0) as lowerk_ss,
               if(prod=3.0, revenue_before3week, 0) as revenue_before3week_ss,
               if(prod=3.0, cpc_before1week, 0) as cpc_before1week_ss,
               if(prod=3.0, charge_before3week, 0) as charge_before3week_ss,
               if(prod=3.0, winningscore_comp12, 0) as winningscore_comp12_ss,
               if(prod=3.0, impr, 0) as impr_ss,
               if(prod=3.0, real_cst_bdgt, 0) as real_cst_bdgt_ss,
               if(prod=3.0, real_aBln, 0) as real_aBln_ss,
               if(prod=3.0, feacomp_ctr, 0) as feacomp_ctr_ss,
               if(prod=3.0, revenue_before2week, 0) as revenue_before2week_ss,
               if(prod=3.0, ratioctr, 0) as ratioctr_ss,
               if(prod=3.0, real_price, 0) as real_price_ss,
               if(prod=3.0, lowerbid, 0) as lowerbid_ss,
               if(prod=3.0, ratio_aimcpc, 0) as ratio_aimcpc_ss,
               if(prod=3.0, cpc_before4week, 0) as cpc_before4week_ss,
               if(prod=3.0, order_num_before3week, 0) as order_num_before3week_ss,
               if(prod=3.0, histctr, 0) as histctr_ss,
               if(prod=3.0, mt_butie_before2week, 0) as mt_butie_before2week_ss,
               if(prod=3.0, mt_butie_before3week, 0) as mt_butie_before3week_ss,
               if(prod=3.0, real_budget, 0) as real_budget_ss,
               if(prod=3.0, order_num_before4week, 0) as order_num_before4week_ss,
               if(prod=3.0, cpc, 0) as cpc_ss,
               if(prod=3.0, order_num_before1week, 0) as order_num_before1week_ss,
               if(prod=3.0, revenue_before1week, 0) as revenue_before1week_ss,
               if(prod=3.0, mt_butie, 0) as mt_butie_ss,
               if(prod=3.0, cpc_before2week, 0) as cpc_before2week_ss,
               if(prod=3.0, cpc_before3week, 0) as cpc_before3week_ss,
               if(prod=3.0, aimcpc, 0) as aimcpc_ss,
               if(prod=3.0, mt_butie_before1week, 0) as mt_butie_before1week_ss,
               if(prod=3.0, real_csm_bln, 0) as real_csm_bln_ss,
               if(prod=3.0, click_before3week, 0) as click_before3week_ss,
               if(prod=3.0, charge_before1week, 0) as charge_before1week_ss,
               if(prod=3.0, click_before4week, 0) as click_before4week_ss,
               if(prod=3.0, click_before2week, 0) as click_before2week_ss,
               if(prod=3.0, winningscore_qs, 0) as winningscore_qs_ss,
               if(prod=3.0, cityid, 0) as cityid_ss,
               if(prod=3.0, ratio_click, 0) as ratio_click_ss,
               if(prod=3.0, ratioctr_exp, 0) as ratioctr_exp_ss,
               if(prod=3.0, upperk, 0) as upperk_ss,
               if(prod=3.0, charge, 0) as charge_ss,
               if(prod=3.0, revenue_before4week, 0) as revenue_before4week_ss,
               if(prod=3.0, ctr, 0) as ctr_ss,
               if(prod=3.0, order_num, 0) as order_num_ss,
               if(prod=3.0, impr_before4week, 0) as impr_before4week_ss,
               if(prod=3.0, impr_before2week, 0) as impr_before2week_ss,
               if(prod=3.0, mt_butie_before4week, 0) as mt_butie_before4week_ss,
               if(prod=3.0, ratio_cpc, 0) as ratio_cpc_ss,
               if(prod=3.0, impr_before3week, 0) as impr_before3week_ss,
               if(prod=3.0, charge_before4week, 0) as charge_before4week_ss,
               if(prod=3.0, real_bln, 0) as real_bln_ss,
               
               if(prod=4.0, user_tag_pref_click_3day, 0) as user_tag_pref_click_3day_push,
               if(prod=4.0, rawbid, 0) as rawbid_push,
               if(prod=4.0, precisebid, 0) as precisebid_push,
               if(prod=4.0, supplybiztype, 0) as supplybiztype_push,
               if(prod=4.0, is_expose, 0) as is_expose_push,
               if(prod=4.0, is_click, 0) as is_click_push,
               if(prod=4.0, final_charge, 0) as final_charge_push,
               if(prod=4.0, is_order, 0) as is_order_push,
               if(prod=4.0, revenue, 0) as revenue_push,
               if(prod=4.0, unused, 0) as unused_push,
               if(prod=4.0, click, 0) as click_push,
               
               if(prod=4.0, hour, 0) as hour_push,
               if(prod=4.0, user_ord_num_weekdays, 0) as user_ord_num_weekdays_push,
               if(prod=4.0, charge_before2week, 0) as charge_before2week_push,
               if(prod=4.0, order_num_before2week, 0) as order_num_before2week_push,
               if(prod=4.0, impr_before1week, 0) as impr_before1week_push,
               if(prod=4.0, feacomp_cvr, 0) as feacomp_cvr_push,
               if(prod=4.0, click_before1week, 0) as click_before1week_push,
               if(prod=4.0, lowerk, 0) as lowerk_push,

               if(prod=4.0, revenue_before3week, 0) as revenue_before3week_push,
               if(prod=4.0, cpc_before1week, 0) as cpc_before1week_push,
               if(prod=4.0, charge_before3week, 0) as charge_before3week_push,
               if(prod=4.0, winningscore_comp12, 0) as winningscore_comp12_push,
               if(prod=4.0, impr, 0) as impr_push,
               if(prod=4.0, real_cst_bdgt, 0) as real_cst_bdgt_push,
               if(prod=4.0, real_aBln, 0) as real_aBln_push,
               if(prod=4.0, feacomp_ctr, 0) as feacomp_ctr_push,
               if(prod=4.0, revenue_before2week, 0) as revenue_before2week_push,
               if(prod=4.0, ratioctr, 0) as ratioctr_push,
               if(prod=4.0, real_price, 0) as real_price_push,
               if(prod=4.0, lowerbid, 0) as lowerbid_push,
               if(prod=4.0, ratio_aimcpc, 0) as ratio_aimcpc_push,
               if(prod=4.0, cpc_before4week, 0) as cpc_before4week_push,
               if(prod=4.0, order_num_before3week, 0) as order_num_before3week_push,
               if(prod=4.0, histctr, 0) as histctr_push,
               if(prod=4.0, mt_butie_before2week, 0) as mt_butie_before2week_push,
               if(prod=4.0, mt_butie_before3week, 0) as mt_butie_before3week_push,
               if(prod=4.0, real_budget, 0) as real_budget_push,
               if(prod=4.0, order_num_before4week, 0) as order_num_before4week_push,
               if(prod=4.0, cpc, 0) as cpc_push,
               if(prod=4.0, order_num_before1week, 0) as order_num_before1week_push,
               if(prod=4.0, revenue_before1week, 0) as revenue_before1week_push,
               if(prod=4.0, mt_butie, 0) as mt_butie_push,
               if(prod=4.0, cpc_before2week, 0) as cpc_before2week_push,
               if(prod=4.0, cpc_before3week, 0) as cpc_before3week_push,
               if(prod=4.0, aimcpc, 0) as aimcpc_push,
               if(prod=4.0, mt_butie_before1week, 0) as mt_butie_before1week_push,
               if(prod=4.0, real_csm_bln, 0) as real_csm_bln_push,
               if(prod=4.0, click_before3week, 0) as click_before3week_push,
               if(prod=4.0, charge_before1week, 0) as charge_before1week_push,
               if(prod=4.0, click_before4week, 0) as click_before4week_push,
               if(prod=4.0, click_before2week, 0) as click_before2week_push,
               if(prod=4.0, winningscore_qs, 0) as winningscore_qs_push,
               if(prod=4.0, cityid, 0) as cityid_push,
               if(prod=4.0, ratio_click, 0) as ratio_click_push,
               if(prod=4.0, ratioctr_exp, 0) as ratioctr_exp_push,
               if(prod=4.0, upperk, 0) as upperk_push,
               if(prod=4.0, charge, 0) as charge_push,
               if(prod=4.0, revenue_before4week, 0) as revenue_before4week_push,
               if(prod=4.0, ctr, 0) as ctr_push,
               if(prod=4.0, order_num, 0) as order_num_push,
               if(prod=4.0, impr_before4week, 0) as impr_before4week_push,
               if(prod=4.0, impr_before2week, 0) as impr_before2week_push,
               if(prod=4.0, mt_butie_before4week, 0) as mt_butie_before4week_push,
               if(prod=4.0, ratio_cpc, 0) as ratio_cpc_push,
               if(prod=4.0, impr_before3week, 0) as impr_before3week_push,
               if(prod=4.0, charge_before4week, 0) as charge_before4week_push,
               if(prod=4.0, real_bln, 0) as real_bln_push,
               
               if(prod=1.0, real_expose, 0) as real_expose_all_dj,
                if(prod=1.0, real_cpc, 0) as real_cpc_all_dj,
                if(prod=1.0, real_click, 0) as real_click_all_dj,
                if(prod=1.0, real_order, 0) as real_order_all_dj,
                if(prod=1.0, real_csm, 0) as real_csm_all_dj,
                if(prod=1.0, real_revenue, 0) as real_revenue_all_dj,
                
                
                if(prod=2.0, real_expose, 0) as real_expose_all_bj,
                if(prod=2.0, real_cpc, 0) as real_cpc_all_bj,
                if(prod=2.0, real_click, 0) as real_click_all_bj,
                if(prod=2.0, real_order, 0) as real_order_all_bj,
                if(prod=2.0, real_csm, 0) as real_csm_all_bj,
                if(prod=2.0, real_revenue, 0) as real_revenue_all_bj,
                
                if(prod=3.0, real_expose, 0) as real_expose_all_ss,
                if(prod=3.0, real_cpc, 0) as real_cpc_all_ss,
                if(prod=3.0, real_click, 0) as real_click_all_ss,
                if(prod=3.0, real_order, 0) as real_order_all_ss,
                if(prod=3.0, real_csm, 0) as real_csm_all_ss,
                if(prod=3.0, real_revenue, 0) as real_revenue_all_ss,
                
                 if(prod=4.0, real_expose, 0) as real_expose_all_push,
                if(prod=4.0, real_cpc, 0) as real_cpc_all_push,
                if(prod=4.0, real_click, 0) as real_click_all_push,
                if(prod=4.0, real_order, 0) as real_order_all_push,
                if(prod=4.0, real_csm, 0) as real_csm_all_push,
                if(prod=4.0, real_revenue, 0) as real_revenue_all_push,	
                
                
                real_cpc_dj,real_impr_dj,real_click_dj,real_order_dj,real_revenue_dj,real_cash_dj,real_charge_dj,
                real_cpc_bj,real_impr_bj,real_click_bj,real_order_bj,real_revenue_bj,real_cash_bj,real_charge_bj,
                real_cpc_ss,real_impr_ss,real_click_ss,real_order_ss,real_revenue_ss,real_cash_ss,real_charge_ss,
                real_cpc_push,real_impr_push,real_click_push,real_order_push,real_revenue_push,real_cash_push,real_charge_push,rlkAct
    	FROM(
                select dt,
                       pvid,
                       user_id,
                       uuid,
                    COALESCE(cast(poi_id as String),0) as poi_id,
                    COALESCE(cast(request_time as bigint),0) as request_time,
                    COALESCE(cast(qsbid as float),0) as rawbid,
                    COALESCE(cast(precisebid as float),0) as precisebid,
                    COALESCE(cast(demandbiztype as float),0) as demandbiztype,
                    COALESCE(cast(supplybiztype as float),0) as supplybiztype,
                    COALESCE(cast(is_expose as float),0) as is_expose,
                    COALESCE(cast(is_click as float),0) as is_click,
                    COALESCE(cast(final_charge as float),0) as final_charge,
                    COALESCE(cast(is_order as float),0) as is_order,
                    COALESCE(cast(revenue as float),0) as revenue,
                    COALESCE(cast(unused as float),0) as unused,
                    COALESCE(cast(click as float),0) as click,
                    COALESCE(cast(user_tag_pref_click_3day as float),0) as user_tag_pref_click_3day,
                    COALESCE(cast(hour as float),0) as hour,
                    COALESCE(cast(user_ord_num_weekdays as float),0) as user_ord_num_weekdays,
                    COALESCE(cast(charge_before2week as float),0) as charge_before2week,
                    COALESCE(cast(order_num_before2week as float),0) as order_num_before2week,
                    COALESCE(cast(impr_before1week as float),0) as impr_before1week,
                    COALESCE(cast(feacomp_cvr as float),0) as feacomp_cvr,
                    COALESCE(cast(click_before1week as float),0) as click_before1week,
                    COALESCE(cast(lowerk as float),0) as lowerk,
                    COALESCE(cast(revenue_before3week as float),0) as revenue_before3week,
                    COALESCE(cast(cpc_before1week as float),0) as cpc_before1week,
                    COALESCE(cast(charge_before3week as float),0) as charge_before3week,
                    COALESCE(cast(winningscore_comp12 as float),0) as winningscore_comp12,
                    COALESCE(cast(impr as float),0) as impr,

                    COALESCE(cast(feacomp_ctr as float),0) as feacomp_ctr,
                    COALESCE(cast(real_cst_bdgt as float),0) as real_cst_bdgt,
                    COALESCE(cast(real_aBln as float),0) as real_aBln,
                    COALESCE(cast(real_price as float),0) as real_price,
                    COALESCE(cast(real_budget as float),0) as real_budget,
                     COALESCE(cast(real_csm_bln as float),0) as real_csm_bln,
                    COALESCE(cast(real_bln as float),0) as real_bln,
                    COALESCE(cast(real_cpc as float),0) as real_cpc,
                    COALESCE(cast(real_click as float),0) as real_click,
                    COALESCE(cast(real_order as float),0) as real_order,
                    COALESCE(cast(real_revenue as float),0) as real_revenue,
                     COALESCE(cast(real_csm as float),0) as real_csm,
                    COALESCE(cast(real_expose as float),0) as real_expose,
        
        
                    COALESCE(cast(real_cpc_dj as float),0) as real_cpc_dj,
                    COALESCE(cast(real_impr_dj as float),0) as real_impr_dj,
                    COALESCE(cast(real_click_dj as float),0) as real_click_dj,
                    COALESCE(cast(real_order_dj as float),0) as real_order_dj,
                    COALESCE(cast(real_revenue_dj as float),0) as real_revenue_dj,
                    COALESCE(cast(real_cash_dj as float),0) as real_cash_dj,
                    COALESCE(cast(real_charge_dj as float),0) as real_charge_dj,

                    COALESCE(cast(real_cpc_bj as float),0) as real_cpc_bj,                
                    COALESCE(cast(real_impr_bj as float),0) as real_impr_bj,
                    COALESCE(cast(real_click_bj as float),0) as real_click_bj,
                    COALESCE(cast(real_order_bj as float),0) as real_order_bj,
                    COALESCE(cast(real_revenue_bj as float),0) as real_revenue_bj,
                    COALESCE(cast(real_cash_bj as float),0) as real_cash_bj,
                    COALESCE(cast(real_charge_bj as float),0) as real_charge_bj,

                    COALESCE(cast(real_cpc_ss as float),0) as real_cpc_ss,
                    COALESCE(cast(real_impr_ss as float),0) as real_impr_ss,
                    COALESCE(cast(real_click_ss as float),0) as real_click_ss,
                    COALESCE(cast(real_order_ss as float),0) as real_order_ss,
                    COALESCE(cast(real_revenue_ss as float),0) as real_revenue_ss,
                    COALESCE(cast(real_cash_ss as float),0) as real_cash_ss,
                    COALESCE(cast(real_charge_ss as float),0) as real_charge_ss,

                    COALESCE(cast(real_cpc_push as float),0) as real_cpc_push,
                    COALESCE(cast(real_impr_push as float),0) as real_impr_push,
                    COALESCE(cast(real_click_push as float),0) as real_click_push,
                    COALESCE(cast(real_order_push as float),0) as real_order_push,
                    COALESCE(cast(real_revenue_push as float),0) as real_revenue_push,
                    COALESCE(cast(real_cash_push as float),0) as real_cash_push,
                    COALESCE(cast(real_charge_push as float),0) as real_charge_push,

                    COALESCE(cast(revenue_before2week as float),0) as revenue_before2week,
                    COALESCE(cast(ratioctr as float),0) as ratioctr,

                    COALESCE(cast(lowerbid as float),0) as lowerbid,
                    COALESCE(cast(ratio_aimcpc as float),0) as ratio_aimcpc,
                    COALESCE(cast(cpc_before4week as float),0) as cpc_before4week,
                    COALESCE(cast(order_num_before3week as float),0) as order_num_before3week,
                    COALESCE(cast(histctr as float),0) as histctr,
                    COALESCE(cast(mt_butie_before2week as float),0) as mt_butie_before2week,
                    COALESCE(cast(mt_butie_before3week as float),0) as mt_butie_before3week,

                    COALESCE(cast(order_num_before4week as float),0) as order_num_before4week,
                    COALESCE(cast(cpc as float),0) as cpc,
                    COALESCE(cast(order_num_before1week as float),0) as order_num_before1week,
                    COALESCE(cast(revenue_before1week as float),0) as revenue_before1week,
                    COALESCE(cast(mt_butie as float),0) as mt_butie,
                    COALESCE(cast(cpc_before2week as float),0) as cpc_before2week,
                    COALESCE(cast(cpc_before3week as float),0) as cpc_before3week,
                    COALESCE(cast(aimcpc as float),0) as aimcpc,
                    COALESCE(cast(mt_butie_before1week as float),0) as mt_butie_before1week,

                    COALESCE(cast(click_before3week as float),0) as click_before3week,
                    COALESCE(cast(charge_before1week as float),0) as charge_before1week,
                    COALESCE(cast(click_before4week as float),0) as click_before4week,
                    COALESCE(cast(click_before2week as float),0) as click_before2week,

                    COALESCE(cast(winningscore_qs as float),0) as winningscore_qs,
                    COALESCE(cast(cityid as float),0) as cityid,
                    COALESCE(cast(ratio_click as float),0) as ratio_click,
                    COALESCE(cast(prod as float),0) as prod,
                    COALESCE(cast(ratioctr_exp as float),0) as ratioctr_exp,
                    COALESCE(cast(upperk as float),0) as upperk,
                    COALESCE(cast(charge as float),0) as charge,
                    COALESCE(cast(revenue_before4week as float),0) as revenue_before4week,
                    COALESCE(cast(ctr as float),0) as ctr,
                    COALESCE(cast(order_num as float),0) as order_num,
                    COALESCE(cast(impr_before4week as float),0) as impr_before4week,
                    COALESCE(cast(impr_before2week as float),0) as impr_before2week,
                    COALESCE(cast(mt_butie_before4week as float),0) as mt_butie_before4week,

                    COALESCE(cast(ratio_cpc as float),0) as ratio_cpc,
                    COALESCE(cast(impr_before3week as float),0) as impr_before3week,
                    COALESCE(cast(charge_before4week as float),0) as charge_before4week,
                    COALESCE(cast(rlkAct as string),0) as rlkAct
                       
                       from ${FPExtendRTFeaturetableName}
                       where delete_res!='1.01' and delete_res!='1.04' 
               )
       )



""")

2023-01-19 10:32:11
执行 
 
SELECT dt,
       pvid,
       user_id,
       uuid,
       poi_id,
       request_time,
       prod,
       rlkAct,
       rawbid_dj + rawbid_bj + rawbid_ss + rawbid_push as rawbid_single,
       precisebid_dj + precisebid_bj + precisebid_ss + precisebid_push as precisebid_single,
       array(user_tag_pref_click_3day_dj, user_tag_pref_click_3day_bj, user_tag_pref_click_3day_ss, user_tag_pref_click_3day_push) as user_tag_pref_click_3day,
       array(rawbid_dj, rawbid_bj, rawbid_ss, rawbid_push) as rawbid,
       array(precisebid_dj, precisebid_bj, precisebid_ss, precisebid_push) as precisebid,
       array(supplybiztype_dj, supplybiztype_bj, supplybiztype_ss, supplybiztype_push) as supplybiztype,
       array(is_expose_dj, is_expose_bj, is_expose_ss, is_expose_push) as is_expose,
       array(is_click_dj, is_click_bj, is_click_ss, is_click_push) as is_click,
       array(final_charge_dj, final_charge_bj, final_charge_ss, final_charge_push) as final_charge,
 

sql at cmd2.sc:66

16000 / 16000

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1674095531474' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2023-01-19 10:35:08


processTable: String = "tmp.AFO_RL_1674095531474"

In [9]:
var trajectoryTableName=generateTrajectory(processTable,Array("dt","poi_id","prod"),Array("request_time","pvid"),Array("dt","poi_id","prod"),"mart_waimaiad")

[Ljava.lang.String;@3f76f237
2023-01-19 10:35:08
执行 
 select * from tmp.AFO_RL_1674095531474 limit 1
2023-01-19 10:35:08
2023-01-19 10:35:08
执行 
 
        select t1.dt,t1.pvid,t1.user_id,t1.uuid,t1.poi_id,t1.request_time,t1.prod,t1.rlkAct,t1.rawbid_single,t1.precisebid_single,t1.user_tag_pref_click_3day,t1.rawbid,t1.precisebid,t1.supplybiztype,t1.is_expose,t1.is_click,t1.final_charge,t1.is_order,t1.revenue,t1.unused,t1.click,t1.hour,t1.user_ord_num_weekdays,t1.charge_before2week,t1.order_num_before2week,t1.impr_before1week,t1.feacomp_cvr,t1.click_before1week,t1.lowerk,t1.revenue_before3week,t1.cpc_before1week,t1.charge_before3week,t1.winningscore_comp12,t1.impr,t1.real_cst_bdgt,t1.real_aBln,t1.feacomp_ctr,t1.real_expose,t1.real_click,t1.real_order,t1.real_cpc,t1.real_csm,t1.real_cash,t1.real_expose_all,t1.real_click_all,t1.real_order_all,t1.real_cpc_all,t1.real_csm_all,t1.real_revenue_all,t1.real_revenue,t1.revenue_before2week,t1.ratioctr,t1.real_price,t1.lowerbid,t1.ratio_aimcpc,t1.cp

sql at cmd2.sc:205

16009 / 16000

sql at cmd2.sc:205

16000 / 16000

...

[Stage 7:======================================================>(199 + 1) / 200]

sql at cmd2.sc:205

200 / 200

Moved: 'viewfs://hadoop-meituan/ghnn07/warehouse/mart_waimaiad.db/afo_rl_1674095531474_trajectory' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current


trajectoryTableName: String = "mart_waimaiad.AFO_RL_1674095531474_trajectory"

In [10]:
// 需要修改的地方：
var tableNameAfterTrajectory=createTableFromSQL(s"""
SELECT *,
       array(action_pre_dj, action_pre_bj, action_pre_ss, action_pre_push) as action
  FROM (
        select *,
             if(prod=1,  case when k_dj<= 0.5            then 0
                     when  0.5<k_dj and k_dj<1.5 then FLOOR((k_dj-0.5)/0.05)
                    else 19 end, 0)  as action_pre_dj,
            if(prod=2, if(rlkAct_bj>-1,rlkAct_bj, case when k_bj<= 0.75            then 0
                     when 0.75<k_bj and k_bj<1.5 then FLOOR((k_bj-0.75)/0.0375)
                    else 19 end),0)  as action_pre_bj,
           if(prod=3,  case when k_ss<= 0.5            then 0
                    when 0.5<k_ss and k_ss<1.5 then FLOOR((k_ss-0.5)/0.05)
                    else 19 end, 0)  as action_pre_ss,
       if(prod=4, if(rlkAct_push>-1,rlkAct_push, case when k_push<= 0.5            then 0
                    when 0.5<k_push and k_push<1.5 then FLOOR((k_push-0.5)/0.05)
                    else 19 end), 0)  as action_pre_push
               
from(
   select dt,poi_id,pvid,rawbid,precisebid,is_expose,is_click,final_charge,is_order,revenue,unused,click,user_tag_pref_click_3day,hour,user_ord_num_weekdays,charge_before2week,order_num_before2week,impr_before1week,feacomp_cvr,click_before1week,lowerk,revenue_before3week,cpc_before1week,charge_before3week,winningscore_comp12,impr,feacomp_ctr,real_expose_all,real_cpc_all,real_csm_all,real_cst_bdgt,real_aBln,real_revenue_all,real_price,real_budget,real_csm_bln,real_click_all,real_order_all,real_bln,revenue_before2week,ratioctr,lowerbid,ratio_aimcpc,cpc_before4week,order_num_before3week,histctr,mt_butie_before2week,mt_butie_before3week,order_num_before4week,cpc,order_num_before1week,revenue_before1week,mt_butie,cpc_before2week,cpc_before3week,aimcpc,mt_butie_before1week,click_before3week,charge_before1week,click_before4week,click_before2week,winningscore_qs,cityid,ratio_click,prod,ratioctr_exp,upperk,charge,revenue_before4week,ctr,order_num,impr_before4week,impr_before2week,mt_butie_before4week,ratio_cpc,impr_before3week,charge_before4week,
   real_cpc,real_csm,real_click,real_cash,real_order,real_revenue,real_expose,   
   if (size(next_real_click)=-1 ,array(0,0,0,0),next_real_click) as next_real_click,if (size(next_real_cpc)=-1 ,array(0,0,0,0),next_real_cpc) as next_real_cpc,if (size(next_real_csm)=-1 ,array(0,0,0,0),next_real_csm) as next_real_csm,if (size(next_real_cash)=-1 ,array(0,0,0,0),next_real_cash) as next_real_cash,if (size(next_real_order)=-1 ,array(0,0,0,0),next_real_order) as next_real_order,if (size(next_real_revenue)=-1 ,array(0,0,0,0),next_real_revenue) as next_real_revenue,if (size(next_real_expose)=-1 ,array(0,0,0,0),next_real_expose) as next_real_expose,
   if (size(next_rawbid)=-1 ,array(0,0,0,0),next_rawbid) as next_rawbid,if (size(next_real_expose_all)=-1 ,array(0,0,0,0),next_real_expose_all) as next_real_expose_all,if (size(next_precisebid)=-1 ,array(0,0,0,0),next_precisebid) as next_precisebid,if (size(next_is_expose)=-1 ,array(0,0,0,0),next_is_expose) as next_is_expose,if (size(next_is_click)=-1 ,array(0,0,0,0),next_is_click) as next_is_click,if (size(next_final_charge)=-1 ,array(0,0,0,0),next_final_charge) as next_final_charge,if (size(next_is_order)=-1 ,array(0,0,0,0),next_is_order) as next_is_order,if (size(next_revenue)=-1 ,array(0,0,0,0),next_revenue) as next_revenue,if (size(next_unused)=-1 ,array(0,0,0,0),next_unused) as next_unused,if (size(next_click)=-1 ,array(0,0,0,0),next_click) as next_click,if (size(next_user_tag_pref_click_3day)=-1 ,array(0,0,0,0),next_user_tag_pref_click_3day) as next_user_tag_pref_click_3day,if (size(next_hour)=-1 ,array(0,0,0,0),next_hour) as next_hour,if (size(next_user_ord_num_weekdays)=-1 ,array(0,0,0,0),next_user_ord_num_weekdays) as next_user_ord_num_weekdays,if (size(next_charge_before2week)=-1 ,array(0,0,0,0),next_charge_before2week) as next_charge_before2week,if (size(next_order_num_before2week)=-1 ,array(0,0,0,0),next_order_num_before2week) as next_order_num_before2week,if (size(next_impr_before1week)=-1 ,array(0,0,0,0),next_impr_before1week) as next_impr_before1week,if (size(next_feacomp_cvr)=-1 ,array(0,0,0,0),next_feacomp_cvr) as next_feacomp_cvr,if (size(next_click_before1week)=-1 ,array(0,0,0,0),next_click_before1week) as next_click_before1week,if (size(next_lowerk)=-1 ,array(0,0,0,0),next_lowerk) as next_lowerk,if (size(next_revenue_before3week)=-1 ,array(0,0,0,0),next_revenue_before3week) as next_revenue_before3week,if (size(next_cpc_before1week)=-1 ,array(0,0,0,0),next_cpc_before1week) as next_cpc_before1week,if (size(next_charge_before3week)=-1 ,array(0,0,0,0),next_charge_before3week) as next_charge_before3week,if (size(next_winningscore_comp12)=-1 ,array(0,0,0,0),next_winningscore_comp12) as next_winningscore_comp12,if (size(next_impr)=-1 ,array(0,0,0,0),next_impr) as next_impr,if (size(next_feacomp_ctr)=-1 ,array(0,0,0,0),next_feacomp_ctr) as next_feacomp_ctr,if (size(next_revenue_before2week)=-1 ,array(0,0,0,0),next_revenue_before2week) as next_revenue_before2week,if (size(next_ratioctr)=-1 ,array(0,0,0,0),next_ratioctr) as next_ratioctr,if (size(next_lowerbid)=-1 ,array(0,0,0,0),next_lowerbid) as next_lowerbid,if (size(next_ratio_aimcpc)=-1 ,array(0,0,0,0),next_ratio_aimcpc) as next_ratio_aimcpc,if (size(next_cpc_before4week)=-1 ,array(0,0,0,0),next_cpc_before4week) as next_cpc_before4week,if (size(next_order_num_before3week)=-1 ,array(0,0,0,0),next_order_num_before3week) as next_order_num_before3week,if (size(next_histctr)=-1 ,array(0,0,0,0),next_histctr) as next_histctr,if (size(next_mt_butie_before2week)=-1 ,array(0,0,0,0),next_mt_butie_before2week) as next_mt_butie_before2week,if (size(next_mt_butie_before3week)=-1 ,array(0,0,0,0),next_mt_butie_before3week) as next_mt_butie_before3week,if (size(next_real_cpc_all)=-1 ,array(0,0,0,0),next_real_cpc_all) as next_real_cpc_all,if (size(next_real_csm_all)=-1 ,array(0,0,0,0),next_real_csm_all) as next_real_csm_all,if (size(next_real_revenue_all)=-1 ,array(0,0,0,0),next_real_revenue_all) as next_real_revenue_all,if (size(next_real_click_all)=-1 ,array(0,0,0,0),next_real_click_all) as next_real_click_all,if (size(next_real_order_all)=-1 ,array(0,0,0,0),next_real_order_all) as next_real_order_all,if (size(next_real_cst_bdgt)=-1 ,array(0,0,0,0),next_real_cst_bdgt) as next_real_cst_bdgt,if (size(next_real_aBln)=-1 ,array(0,0,0,0),next_real_aBln) as next_real_aBln,if (size(next_real_price)=-1 ,array(0,0,0,0),next_real_price) as next_real_price,if (size(next_real_budget)=-1 ,array(0,0,0,0),next_real_budget) as next_real_budget,if (size(next_real_csm_bln)=-1 ,array(0,0,0,0),next_real_csm_bln) as next_real_csm_bln,if (size(next_real_bln)=-1 ,array(0,0,0,0),next_real_bln) as next_real_bln,if (size(next_order_num_before4week)=-1 ,array(0,0,0,0),next_order_num_before4week) as next_order_num_before4week,if (size(next_cpc)=-1 ,array(0,0,0,0),next_cpc) as next_cpc,if (size(next_order_num_before1week)=-1 ,array(0,0,0,0),next_order_num_before1week) as next_order_num_before1week,if (size(next_revenue_before1week)=-1 ,array(0,0,0,0),next_revenue_before1week) as next_revenue_before1week,if (size(next_mt_butie)=-1 ,array(0,0,0,0),next_mt_butie) as next_mt_butie,if (size(next_cpc_before2week)=-1 ,array(0,0,0,0),next_cpc_before2week) as next_cpc_before2week,if (size(next_cpc_before3week)=-1 ,array(0,0,0,0),next_cpc_before3week) as next_cpc_before3week,if (size(next_aimcpc)=-1 ,array(0,0,0,0),next_aimcpc) as next_aimcpc,if (size(next_mt_butie_before1week)=-1 ,array(0,0,0,0),next_mt_butie_before1week) as next_mt_butie_before1week,if (size(next_click_before3week)=-1 ,array(0,0,0,0),next_click_before3week) as next_click_before3week,if (size(next_charge_before1week)=-1 ,array(0,0,0,0),next_charge_before1week) as next_charge_before1week,if (size(next_click_before4week)=-1 ,array(0,0,0,0),next_click_before4week) as next_click_before4week,if (size(next_click_before2week)=-1 ,array(0,0,0,0),next_click_before2week) as next_click_before2week,if (size(next_winningscore_qs)=-1 ,array(0,0,0,0),next_winningscore_qs) as next_winningscore_qs,if (size(next_cityid)=-1 ,array(0,0,0,0),next_cityid) as next_cityid,if (size(next_ratio_click)=-1 ,array(0,0,0,0),next_ratio_click) as next_ratio_click,if (size(next_ratioctr_exp)=-1 ,array(0,0,0,0),next_ratioctr_exp) as next_ratioctr_exp,if (size(next_upperk)=-1 ,array(0,0,0,0),next_upperk) as next_upperk,if (size(next_charge)=-1 ,array(0,0,0,0),next_charge) as next_charge,if (size(next_revenue_before4week)=-1 ,array(0,0,0,0),next_revenue_before4week) as next_revenue_before4week,if (size(next_ctr)=-1 ,array(0,0,0,0),next_ctr) as next_ctr,if (size(next_order_num)=-1 ,array(0,0,0,0),next_order_num) as next_order_num,if (size(next_impr_before4week)=-1 ,array(0,0,0,0),next_impr_before4week) as next_impr_before4week,if (size(next_impr_before2week)=-1 ,array(0,0,0,0),next_impr_before2week) as next_impr_before2week,if (size(next_mt_butie_before4week)=-1 ,array(0,0,0,0),next_mt_butie_before4week) as next_mt_butie_before4week,if (size(next_ratio_cpc)=-1 ,array(0,0,0,0),next_ratio_cpc) as next_ratio_cpc,if (size(next_impr_before3week)=-1 ,array(0,0,0,0),next_impr_before3week) as next_impr_before3week,if (size(next_charge_before4week)=-1 ,array(0,0,0,0),next_charge_before4week) as next_charge_before4week,
   rlkAct,
   if(prod=1,precisebid_single/rawbid_single, 0) as k_dj,
   if(prod=2,precisebid_single/(rawbid_single*histctr[1]), 0) as k_bj,
   if(prod=3,precisebid_single/rawbid_single, 0) as k_ss,
   if(prod=4,precisebid_single/(rawbid_single*histctr[3]), 0) as k_push,
   
    cast(if(prod=1,if(rlkAct!='unknown',if(cast(rlkAct as float)>19,19,rlkAct),-10),0) as INT) as rlkAct_dj,
    cast(if(prod=2,if(rlkAct!='unknown',if(cast(rlkAct as float)>19,19,rlkAct),-10),0) as INT)  as rlkAct_bj,
    cast(if(prod=3,if(rlkAct!='unknown',if(cast(rlkAct as float)>19,19,rlkAct),-10),0) as INT) as rlkAct_ss,
    cast(if(prod=4,if(rlkAct!='unknown',if(cast(rlkAct as float)>19,19,rlkAct),-10),0) as INT)  as rlkAct_push,
    
   is_click as reward, cast(if(next_poi_id is null, 1.0, 0.0) as float) as is_terminal
    from ${trajectoryTableName}
    where 
    next_rawbid[0] is not null and next_precisebid[0] is not null and next_is_expose[0] is not null and next_is_click[0] is not null and next_final_charge[0] is not null and next_is_order[0] is not null and next_revenue[0] is not null and next_unused[0] is not null and next_click[0] is not null and next_user_tag_pref_click_3day[0] is not null and next_hour[0] is not null and next_user_ord_num_weekdays[0] is not null and next_charge_before2week[0] is not null and next_order_num_before2week[0] is not null and next_impr_before1week[0] is not null and next_feacomp_cvr[0] is not null and next_click_before1week[0] is not null and next_lowerk[0] is not null and next_revenue_before3week[0] is not null and next_cpc_before1week[0] is not null and next_charge_before3week[0] is not null and next_winningscore_comp12[0] is not null and next_impr[0] is not null and next_feacomp_ctr[0] is not null and next_revenue_before2week[0] is not null and next_ratioctr[0] is not null and next_lowerbid[0] is not null and next_ratio_aimcpc[0] is not null and next_cpc_before4week[0] is not null and next_order_num_before3week[0] is not null and next_histctr[0] is not null and next_mt_butie_before2week[0] is not null and next_mt_butie_before3week[0] is not null and next_real_cpc_all[0] is not null and next_real_csm_all[0] is not null and next_real_revenue_all[0] is not null and next_real_click_all[0] is not null and next_real_order_all[0] is not null and next_real_cst_bdgt[0] is not null and next_real_aBln[0] is not null and next_real_price[0] is not null and next_real_budget[0] is not null and next_real_csm_bln[0] is not null and next_real_bln[0] is not null and next_order_num_before4week[0] is not null and next_cpc[0] is not null and next_order_num_before1week[0] is not null and next_revenue_before1week[0] is not null and next_mt_butie[0] is not null and next_cpc_before2week[0] is not null and next_cpc_before3week[0] is not null and next_aimcpc[0] is not null and next_mt_butie_before1week[0] is not null and next_click_before3week[0] is not null and next_charge_before1week[0] is not null and next_click_before4week[0] is not null and next_click_before2week[0] is not null and next_winningscore_qs[0] is not null and next_cityid[0] is not null and next_ratio_click[0] is not null and next_ratioctr_exp[0] is not null and next_upperk[0] is not null and next_charge[0] is not null and next_revenue_before4week[0] is not null and next_ctr[0] is not null and next_order_num[0] is not null and next_impr_before4week[0] is not null and next_impr_before2week[0] is not null and next_mt_butie_before4week[0] is not null and next_ratio_cpc[0] is not null and next_impr_before3week[0] is not null and next_charge_before4week[0] is not null
    and next_rawbid[1] is not null and next_precisebid[1] is not null and next_is_expose[1] is not null and next_is_click[1] is not null and next_final_charge[1] is not null and next_is_order[1] is not null and next_revenue[1] is not null and next_unused[1] is not null and next_click[1] is not null and next_user_tag_pref_click_3day[1] is not null and next_hour[1] is not null and next_user_ord_num_weekdays[1] is not null and next_charge_before2week[1] is not null and next_order_num_before2week[1] is not null and next_impr_before1week[1] is not null and next_feacomp_cvr[1] is not null and next_click_before1week[1] is not null and next_lowerk[1] is not null and next_revenue_before3week[1] is not null and next_cpc_before1week[1] is not null and next_charge_before3week[1] is not null and next_winningscore_comp12[1] is not null and next_impr[1] is not null and next_feacomp_ctr[1] is not null and next_revenue_before2week[1] is not null and next_ratioctr[1] is not null and next_lowerbid[1] is not null and next_ratio_aimcpc[1] is not null and next_cpc_before4week[1] is not null and next_order_num_before3week[1] is not null and next_histctr[1] is not null and next_mt_butie_before2week[1] is not null and next_mt_butie_before3week[1] is not null and next_real_cpc_all[1] is not null and next_real_csm_all[1] is not null and next_real_revenue_all[1] is not null and next_real_click_all[1] is not null and next_real_order_all[1] is not null and next_real_cst_bdgt[1] is not null and next_real_aBln[1] is not null and next_real_price[1] is not null and next_real_budget[1] is not null and next_real_csm_bln[1] is not null and next_real_bln[1] is not null and next_order_num_before4week[1] is not null and next_cpc[1] is not null and next_order_num_before1week[1] is not null and next_revenue_before1week[1] is not null and next_mt_butie[1] is not null and next_cpc_before2week[1] is not null and next_cpc_before3week[1] is not null and next_aimcpc[1] is not null and next_mt_butie_before1week[1] is not null and next_click_before3week[1] is not null and next_charge_before1week[1] is not null and next_click_before4week[1] is not null and next_click_before2week[1] is not null and next_winningscore_qs[1] is not null and next_cityid[1] is not null and next_ratio_click[1] is not null and next_ratioctr_exp[1] is not null and next_upperk[1] is not null and next_charge[1] is not null and next_revenue_before4week[1] is not null and next_ctr[1] is not null and next_order_num[1] is not null and next_impr_before4week[1] is not null and next_impr_before2week[1] is not null and next_mt_butie_before4week[1] is not null and next_ratio_cpc[1] is not null and next_impr_before3week[1] is not null and next_charge_before4week[1] is not null
    and next_rawbid[2] is not null and next_precisebid[2] is not null and next_is_expose[2] is not null and next_is_click[2] is not null and next_final_charge[2] is not null and next_is_order[2] is not null and next_revenue[2] is not null and next_unused[2] is not null and next_click[2] is not null and next_user_tag_pref_click_3day[2] is not null and next_hour[2] is not null and next_user_ord_num_weekdays[2] is not null and next_charge_before2week[2] is not null and next_order_num_before2week[2] is not null and next_impr_before1week[2] is not null and next_feacomp_cvr[2] is not null and next_click_before1week[2] is not null and next_lowerk[2] is not null and next_revenue_before3week[2] is not null and next_cpc_before1week[2] is not null and next_charge_before3week[2] is not null and next_winningscore_comp12[2] is not null and next_impr[2] is not null and next_feacomp_ctr[2] is not null and next_revenue_before2week[2] is not null and next_ratioctr[2] is not null and next_lowerbid[2] is not null and next_ratio_aimcpc[2] is not null and next_cpc_before4week[2] is not null and next_order_num_before3week[2] is not null and next_histctr[2] is not null and next_mt_butie_before2week[2] is not null and next_mt_butie_before3week[2] is not null and next_real_cpc_all[2] is not null and next_real_csm_all[2] is not null and next_real_revenue_all[2] is not null and next_real_click_all[2] is not null and next_real_order_all[2] is not null and next_real_cst_bdgt[2] is not null and next_real_aBln[2] is not null and next_real_price[2] is not null and next_real_budget[2] is not null and next_real_csm_bln[2] is not null and next_real_bln[2] is not null and next_order_num_before4week[2] is not null and next_cpc[2] is not null and next_order_num_before1week[2] is not null and next_revenue_before1week[2] is not null and next_mt_butie[2] is not null and next_cpc_before2week[2] is not null and next_cpc_before3week[2] is not null and next_aimcpc[2] is not null and next_mt_butie_before1week[2] is not null and next_click_before3week[2] is not null and next_charge_before1week[2] is not null and next_click_before4week[2] is not null and next_click_before2week[2] is not null and next_winningscore_qs[2] is not null and next_cityid[2] is not null and next_ratio_click[2] is not null and next_ratioctr_exp[2] is not null and next_upperk[2] is not null and next_charge[2] is not null and next_revenue_before4week[2] is not null and next_ctr[2] is not null and next_order_num[2] is not null and next_impr_before4week[2] is not null and next_impr_before2week[2] is not null and next_mt_butie_before4week[2] is not null and next_ratio_cpc[2] is not null and next_impr_before3week[2] is not null and next_charge_before4week[2] is not null
    and next_rawbid[3] is not null and next_precisebid[3] is not null and next_is_expose[3] is not null and next_is_click[3] is not null and next_final_charge[3] is not null and next_is_order[3] is not null and next_revenue[3] is not null and next_unused[3] is not null and next_click[3] is not null and next_user_tag_pref_click_3day[3] is not null and next_hour[3] is not null and next_user_ord_num_weekdays[3] is not null and next_charge_before2week[3] is not null and next_order_num_before2week[3] is not null and next_impr_before1week[3] is not null and next_feacomp_cvr[3] is not null and next_click_before1week[3] is not null and next_lowerk[3] is not null and next_revenue_before3week[3] is not null and next_cpc_before1week[3] is not null and next_charge_before3week[3] is not null and next_winningscore_comp12[3] is not null and next_impr[3] is not null and next_feacomp_ctr[3] is not null and next_revenue_before2week[3] is not null and next_ratioctr[3] is not null and next_lowerbid[3] is not null and next_ratio_aimcpc[3] is not null and next_cpc_before4week[3] is not null and next_order_num_before3week[3] is not null and next_histctr[3] is not null and next_mt_butie_before2week[3] is not null and next_mt_butie_before3week[3] is not null and next_real_cpc_all[3] is not null and next_real_csm_all[3] is not null and next_real_revenue_all[3] is not null and next_real_click_all[3] is not null and next_real_order_all[3] is not null and next_real_cst_bdgt[3] is not null and next_real_aBln[3] is not null and next_real_price[3] is not null and next_real_budget[3] is not null and next_real_csm_bln[3] is not null and next_real_bln[3] is not null and next_order_num_before4week[3] is not null and next_cpc[3] is not null and next_order_num_before1week[3] is not null and next_revenue_before1week[3] is not null and next_mt_butie[3] is not null and next_cpc_before2week[3] is not null and next_cpc_before3week[3] is not null and next_aimcpc[3] is not null and next_mt_butie_before1week[3] is not null and next_click_before3week[3] is not null and next_charge_before1week[3] is not null and next_click_before4week[3] is not null and next_click_before2week[3] is not null and next_winningscore_qs[3] is not null and next_cityid[3] is not null and next_ratio_click[3] is not null and next_ratioctr_exp[3] is not null and next_upperk[3] is not null and next_charge[3] is not null and next_revenue_before4week[3] is not null and next_ctr[3] is not null and next_order_num[3] is not null and next_impr_before4week[3] is not null and next_impr_before2week[3] is not null and next_mt_butie_before4week[3] is not null and next_ratio_cpc[3] is not null and next_impr_before3week[3] is not null and next_charge_before4week[3] is not null
    and next_real_cpc[0] is not null and next_real_cpc[1] is not null and next_real_cpc[2] is not null and next_real_cpc[3] is not null 
    and next_real_csm[0] is not null and next_real_csm[1] is not null and next_real_csm[2] is not null and next_real_csm[3] is not null 
    and next_real_click[0] is not null and next_real_click[1] is not null and next_real_click[2] is not null and next_real_click[3] is not null 
    and next_real_cash[0] is not null and next_real_cash[1] is not null and next_real_cash[2] is not null and next_real_cash[3] is not null 
    and next_real_order[0] is not null and next_real_order[1] is not null and next_real_order[2] is not null and next_real_order[3] is not null 
    and next_real_revenue[0] is not null and next_real_revenue[1] is not null and next_real_revenue[2] is not null and next_real_revenue[3] is not null 
    and next_real_expose[0] is not null and next_real_expose[1] is not null and next_real_expose[2] is not null and next_real_expose[3] is not null 





    )t1)
"""
 ,"mart_waimaiad")


2023-01-19 10:54:18
执行 
 
SELECT *,
       array(action_pre_dj, action_pre_bj, action_pre_ss, action_pre_push) as action
  FROM (
        select *,
             if(prod=1,  case when k_dj<= 0.5            then 0
                     when  0.5<k_dj and k_dj<1.5 then FLOOR((k_dj-0.5)/0.05)
                    else 19 end, 0)  as action_pre_dj,
            if(prod=2, if(rlkAct_bj>-1,rlkAct_bj, case when k_bj<= 0.75            then 0
                     when 0.75<k_bj and k_bj<1.5 then FLOOR((k_bj-0.75)/0.0375)
                    else 19 end),0)  as action_pre_bj,
           if(prod=3,  case when k_ss<= 0.5            then 0
                    when 0.5<k_ss and k_ss<1.5 then FLOOR((k_ss-0.5)/0.05)
                    else 19 end, 0)  as action_pre_ss,
       if(prod=4, if(rlkAct_push>-1,rlkAct_push, case when k_push<= 0.5            then 0
                    when 0.5<k_push and k_push<1.5 then FLOOR((k_push-0.5)/0.05)
                    else 19 end), 0)  as action_pre_push
           

sql at cmd2.sc:66

200 / 200

Moved: 'viewfs://hadoop-meituan/ghnn07/warehouse/mart_waimaiad.db/afo_rl_1674096858087' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2023-01-19 11:17:52


tableNameAfterTrajectory: String = "mart_waimaiad.AFO_RL_1674096858087"

In [ ]:

var tmpTrainTable = createTableFromSQL(s"""
select * from ${tableNameAfterTrajectory}
where dt>=${fp_log_date_start}
and dt<= ${fp_log_date_end} 
""")


2023-01-19 11:17:53
执行 
 
select * from mart_waimaiad.AFO_RL_1674096858087
where dt>=20230111
and dt<= 20230118 

2023-01-19 11:17:53
2023-01-19 11:17:53
执行 
 drop table if exists tmp.AFO_RL_1674098273193
2023-01-19 11:17:53
2023-01-19 11:17:53
执行 
 create table if not exists tmp.AFO_RL_1674098273193(dt String,poi_id String,pvid String,rawbid Array<Float>,precisebid Array<Float>,is_expose Array<Float>,is_click Array<Float>,final_charge Array<Float>,is_order Array<Float>,revenue Array<Float>,unused Array<Float>,click Array<Float>,user_tag_pref_click_3day Array<Float>,hour Array<Float>,user_ord_num_weekdays Array<Float>,charge_before2week Array<Float>,order_num_before2week Array<Float>,impr_before1week Array<Float>,feacomp_cvr Array<Float>,click_before1week Array<Float>,lowerk Array<Float>,revenue_before3week Array<Float>,cpc_before1week Array<Float>,charge_before3week Array<Float>,winningscore_comp12 Array<Float>,impr Array<Float>,feacomp_ctr Array<Float>,real_expose_all Array<Float>,re

sql at cmd2.sc:66 
 (kill)

0 + 200 / 200

[Stage 12:>                                                     (0 + 200) / 200]

In [ ]:
var finalTrainTable= createPTTableFromTableWithSpace(tmpTrainTable,"mart_waimaiad") //正式使用请按照自己的逻辑生成，表的主键是pvid，poi_id

In [ ]:

var tmpPredictTable = createTableFromSQL(s"""
select *
from ${tableNameAfterTrajectory}
where dt = ${predict_dt}
and abs(hash(pvid)) % 100 < 10 
""")

In [ ]:
var finalPredictTable = createPTTableFromTableWithSpace(tmpPredictTable,"mart_waimaiad") //正式使用请按照自己的逻辑生成，表的主键是pvid，poi_id

In [ ]:
trajectoryTableName
finalTrainTable
finalPredictTable

In [ ]:
showTopNFromSQL(s"""
select count(*) from  ${finalTrainTable}
""",1)

In [ ]:
showTopNFromSQL(s"""
select dt,pt from  ${finalTrainTable}
""",1)

In [ ]:
showTopNFromSQL(s"""
select count(*) from  ${finalPredictTable}
""",1)

In [ ]:
showTopNFromSQL(s"""
select dt,pt from ${finalPredictTable}
""",1)

In [ ]:
showTopNFromSQL(s"""
select count(*),action[0] from ${finalTrainTable} where prod=1 group by action[0] order by action[0]
""",30)

In [ ]:
showTopNFromSQL(s"""
select count(*),action[1] from  ${finalTrainTable} where prod=2 group by action[1] order by action[1]
""",30)

In [ ]:
showTopNFromSQL(s"""
select count(*),action[2] from  ${finalTrainTable} where prod=3 group by action[2] order by action[2]
""",30)

In [ ]:
showTopNFromSQL(s"""
select count(*),action[3] from  ${finalTrainTable} where prod=4 group by action[3] order by action[3]
""",30)